In [14]:
import pandas as pd
d1 = pd.read_csv(r"C:\Users\jungi\startcoding\실전편\2\game_player\2020k리그1_성남.csv",encoding = 'cp949')
d1.head()

,Rnd,대회,H/A,득점,도움,슈팅,유효 슈팅,블락된슈팅,벗어난슈팅,PA내 슈팅,...,클리어링,인터셉트,차단,획득,블락,볼미스,파울,피파울,경고,퇴장
0,1,광주vs성남,AWAY,2,2,6,4,1,1,4,...,27,8,84,84,4,6,19,9,1,0
1,2,성남vs인천,HOME,0,0,8,0,1,7,5,...,9,28,37,78,1,6,14,9,0,0
2,3,강원vs성남,AWAY,1,0,24,7,9,8,13,...,15,19,29,81,4,3,16,12,1,0
3,4,서울vs성남,AWAY,1,0,6,4,1,1,3,...,27,21,31,64,1,3,8,18,1,0
4,5,성남vs대구,HOME,1,0,5,2,2,1,3,...,22,20,34,69,6,10,5,15,0,0


In [30]:
import pandas as pd
import glob
import os
import re

# 메인 폴더 경로
base_folder_path = "C:/Users/jungi/startcoding/실전편/2/game_player/2020_player"

# 팀 목록 (팀 폴더 이름)
teams = ['강원', '광주', '대구', '부산','상주','서울','성남', '수원', '울산', '인천', '전북', '포항']

# 사용한 컬럼 목록
columns_to_include = ['포지션', '출전시간(분)', '득점', '인터셉트', '클리어링', '경합 공중 성공', '태클 성공', '유효 슈팅', '키패스', '패스 성공%', '드리블 성공', '슈팅', '크로스 성공']

# 모든 팀에 대한 데이터프레임을 저장할 리스트
all_team_dataframes = []

# 각 팀 폴더를 순회하며 데이터 처리
for team in teams:

    folder_path = os.path.join(base_folder_path, team)
    folder_path = os.path.normpath(folder_path)
    # 특정 패턴과 일치하는 모든 파일 경로 가져오기
    csv_files = glob.glob(os.path.join(folder_path, "2020k리그1_*.csv"))
   
    # 라운드별 데이터프레임을 저장할 딕셔너리
    dataframes = {}

    # 정규표현식 패턴 (예: "2020k리그1_강원_10R _ 전북(A).csv")
    pattern = re.compile(r"2020k리그1_.*_(\d+)R\s.*\.csv")

    # 각 CSV 파일을 읽어서 딕셔너리에 저장
    for file in csv_files:
        match = pattern.match(os.path.basename(file))
        if match:
            round_number = match.group(1)  # 라운드 번호 추출
            dataframes[round_number] = pd.read_csv(file, encoding='cp949')
        else:
            print(f"No match for file: {file}")

    # 메인 데이터프레임 읽기
    main_file_path = os.path.join('C:/Users/jungi/startcoding/실전편/2/game_player', f'2020k리그1_{team}.csv')
    if not os.path.exists(main_file_path):
        print(f"File not found: {main_file_path}")
        continue
    encodings = ['utf-8', 'cp949']

    for encoding in encodings:
        try:
            main_df = pd.read_csv(main_file_path, encoding=encoding)
            print(f"Successfully read the file with encoding: {encoding}")
            break
        except UnicodeDecodeError:
            print(f"Encoding {encoding} failed. Trying next encoding...")
        except FileNotFoundError:
            print(f"File not found: {main_file_path}")
            break
    print(main_df)
    # 새로운 컬럼을 추가할 준비 (기존 컬럼 이름에 접두어를 추가하여 충돌을 피함)
    for col in columns_to_include:
        main_df[f'Player_Stats_{col}'] = [{} for _ in range(len(main_df))]

    # 각 라운드 데이터프레임에서 선수명과 선택된 컬럼들을 딕셔너리로 만들어 메인 데이터프레임에 추가
    for round_number, df in dataframes.items():
        if '선수명' in df.columns:
            print(df)
            for col in columns_to_include:
                if col in df.columns:
                    if col == '포지션':
                        # 포지션은 문자열로 처리
                        player_stats_dict = dict(zip(df['선수명'], df[col]))
                    else:
                        # 비정상적인 값은 0으로 처리하고 숫자형으로 변환
                        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
                        player_stats_dict = dict(zip(df['선수명'], df[col]))
                    # 디버깅: 각 라운드와 컬럼에 대해 생성된 딕셔너리 출력
                    print(f"Round: {round_number}, Column: {col}, Player Stats Dict: {player_stats_dict}")
                    # 메인 데이터프레임에서 해당 라운드에 맞는 행의 해당 컬럼에 딕셔너리 추가
                    main_df.loc[main_df['Rnd'].astype(str) == str(round_number), f'Player_Stats_{col}'] = main_df.loc[main_df['Rnd'].astype(str) == str(round_number), f'Player_Stats_{col}'].apply(lambda x: player_stats_dict)

    # 팀 이름을 추가한 컬럼을 생성
    main_df['Team'] = team

    # 팀 데이터프레임을 리스트에 추가
    all_team_dataframes.append(main_df)

# 모든 팀 데이터프레임을 하나로 합침
combined_df = pd.concat(all_team_dataframes, ignore_index=True)


#상대팀생성
for i in range(len(combined_df)):
    teams=combined_df.loc[i,'대회'].split('vs')
    print(teams)
    if combined_df.loc[i,'Team']==teams[0]:
        combined_df.loc[i,'상대팀'] = teams[1]
    else:
        combined_df.loc[i,'상대팀'] = teams[0]
#득점 실점 기록

for i in range(len(combined_df)):
    temp=combined_df.loc[i,['Rnd','Team','상대팀']]
    our_team=temp[1]
    opponent_team=temp[2]
    Round = temp[0]
    opponent_points = combined_df.loc[(combined_df['Rnd']==Round)&(combined_df['Team']==opponent_team),'득점'].values[0]
    combined_df.loc[i,'실점'] = opponent_points

def result(row):
    if row['득점'] > row['실점']:
        return '승'
    elif row['득점'] == row['실점']:
        return '무'
    else:
        return '패'
combined_df['결과'] = combined_df.apply(result,axis = 1)
combined_df

combined_df.to_csv('2020k1리그종합.csv',encoding = 'cp949', index = False)

Encoding utf-8 failed. Trying next encoding...
Successfully read the file with encoding: cp949
    Rnd      대회   H/A  득점  도움  슈팅  유효 슈팅  블락된슈팅  벗어난슈팅  PA내 슈팅  ...  클리어링   
0     1  강원vs서울  HOME   3   3   8      4      2      2       4  ...    15  \
1     2  상주vs강원  AWAY   0   0  13      1      5      7       9  ...    16   
2     3  강원vs성남  HOME   1   1  12      5      4      3       9  ...    15   
3     4  강원vs전북  HOME   1   1   6      3      1      2       2  ...    20   
4     5  인천vs강원  AWAY   2   0  12      6      5      1       7  ...     8   
5     6  수원vs강원  AWAY   2   2   9      4      1      4       5  ...    14   
6     7  강원vs울산  HOME   0   0  11      4      3      4       3  ...    11   
7     8  포항vs강원  AWAY   0   0   9      1      3      5       6  ...     9   
8     9  대구vs강원  AWAY   1   0  14      4      2      8       7  ...    12   
9    10  강원vs부산  HOME   2   1  21      6      6      9      13  ...    15   
10   11  강원vs광주  HOME   4   4  19      6      4      9    

In [29]:
pd.set_option('display.max_rows', None)
combined_df[['Rnd','대회']]

,Rnd,대회
0,1.0,강원vs서울
1,2.0,상주vs강원
2,3.0,강원vs성남
3,4.0,강원vs전북
4,5.0,인천vs강원
5,6.0,수원vs강원
6,7.0,강원vs울산
7,8.0,포항vs강원
8,9.0,대구vs강원
9,10.0,강원vs부산


In [5]:
import pandas as pd
import glob
import os
import re

# 메인 폴더 경로
base_folder_path = "C:/Users/jungi/startcoding/실전편/2/2024 선수별/"

# 팀 목록 (팀 폴더 이름)
teams = ['강원', '광주', '김천','수원FC', '대구', '대전','서울', '울산', '인천', '전북','제주', '포항']

# 사용한 컬럼 목록
columns_to_include = ['포지션', '출전시간(분)', '득점', '인터셉트', '클리어링', '경합 공중 성공', '태클 성공', '유효 슈팅', '키패스', '패스 성공%', '드리블 성공', '슈팅', '크로스 성공']

# 모든 팀에 대한 데이터프레임을 저장할 리스트
all_team_dataframes = []

# 각 팀 폴더를 순회하며 데이터 처리
for team in teams:

    folder_path = os.path.join(base_folder_path, team)
    folder_path = os.path.normpath(folder_path)
    # 특정 패턴과 일치하는 모든 파일 경로 가져오기
    csv_files = glob.glob(os.path.join(folder_path, "2024k리그1_*.csv"))
   
    # 라운드별 데이터프레임을 저장할 딕셔너리
    dataframes = {}

    # 정규표현식 패턴 (예: "2023k리그1_강원_10R _ 전북(A).csv")
    pattern = re.compile(r"2024k리그1_.*_(\d+)R\s.*\.csv")

    # 각 CSV 파일을 읽어서 딕셔너리에 저장
    for file in csv_files:
        match = pattern.match(os.path.basename(file))
        if match:
            round_number = match.group(1)  # 라운드 번호 추출
            dataframes[round_number] = pd.read_csv(file, encoding='cp949')
        else:
            print(f"No match for file: {file}")

    # 메인 데이터프레임 읽기
    main_file_path = os.path.join('C:/Users/jungi/startcoding/실전편/2', f'2024k리그1_{team}.csv')
    if not os.path.exists(main_file_path):
        print(f"File not found: {main_file_path}")
        continue
    main_df = pd.read_csv(main_file_path,encoding='cp949')

    # 새로운 컬럼을 추가할 준비 (기존 컬럼 이름에 접두어를 추가하여 충돌을 피함)
    for col in columns_to_include:
        main_df[f'Player_Stats_{col}'] = [{} for _ in range(len(main_df))]

    # 각 라운드 데이터프레임에서 선수명과 선택된 컬럼들을 딕셔너리로 만들어 메인 데이터프레임에 추가
    for round_number, df in dataframes.items():
        if '선수명' in df.columns:
            print(df)
            for col in columns_to_include:
                if col in df.columns:
                    if col == '포지션':
                        # 포지션은 문자열로 처리
                        player_stats_dict = dict(zip(df['선수명'], df[col]))
                    else:
                        # 비정상적인 값은 0으로 처리하고 숫자형으로 변환
                        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
                        player_stats_dict = dict(zip(df['선수명'], df[col]))
                    # 디버깅: 각 라운드와 컬럼에 대해 생성된 딕셔너리 출력
                    print(f"Round: {round_number}, Column: {col}, Player Stats Dict: {player_stats_dict}")
                    # 메인 데이터프레임에서 해당 라운드에 맞는 행의 해당 컬럼에 딕셔너리 추가
                    main_df.loc[main_df['Rnd'].astype(str) == str(round_number), f'Player_Stats_{col}'] = main_df.loc[main_df['Rnd'].astype(str) == str(round_number), f'Player_Stats_{col}'].apply(lambda x: player_stats_dict)

    # 팀 이름을 추가한 컬럼을 생성
    main_df['Team'] = team

    # 팀 데이터프레임을 리스트에 추가
    all_team_dataframes.append(main_df)

# 모든 팀 데이터프레임을 하나로 합침
combined_df = pd.concat(all_team_dataframes, ignore_index=True)


#상대팀생성
for i in range(len(combined_df)):
    teams=combined_df.loc[i,'대회'].split('vs')
    if combined_df.loc[i,'Team']==teams[0]:
        combined_df.loc[i,'상대팀'] = teams[1]
    else:
        combined_df.loc[i,'상대팀'] = teams[0]
#득점 실점 기록
for i in range(len(combined_df)):
    temp=combined_df.loc[i,['Rnd','Team','상대팀']]
    our_team=temp[1]
    opponent_team=temp[2]
    Round = temp[0]
    opponent_points = combined_df.loc[(combined_df['Rnd']==Round)&(combined_df['Team']==opponent_team),'득점'].values[0]
    combined_df.loc[i,'실점'] = opponent_points

def result(row):
    if row['득점'] > row['실점']:
        return '승'
    elif row['득점'] == row['실점']:
        return '무'
    else:
        return '패'
combined_df['결과'] = combined_df.apply(result,axis = 1)
combined_df
combined_df.to_csv('2024k1리그종합.csv',encoding = 'cp949', index = False)

    No.  선수명 포지션  등번호 출전시간(분)  득점  도움  슈팅  유효 슈팅  블락된슈팅  ...  클리어링  인터셉트  차단   
0     1  박청효  GK   21      98   0   0   0      0      0  ...     0     3   0  \
1     2  강투지  DF   74      98   0   0   2      1      0  ...     6     2   1   
2     3  윤석영  DF   20      98   0   0   0      0      0  ...     1     1   2   
3     4  이기혁  DF   13      47   0   0   0      0      0  ...     1     0   0   
4     5  황문기  DF   88      96   0   0   0      0      0  ...     0     4   0   
5     6  김강국  MF   18      76   0   0   1      0      0  ...     0     1   6   
6     7  양민혁  MF   47      98   1   0   2      1      0  ...     0     2   1   
7     8  김이석  MF    6      98   0   1   3      2      0  ...     1     1   0   
8     9  유인수  MF   17      57   0   0   2      1      0  ...     1     0   0   
9    10   야고  FW   99      98   0   0   3      0      0  ...     0     1   1   
10   11  이상헌  FW   22      76   0   0   3      0      0  ...     0     0   0   
11   12  김영빈  대기    2      51   0   0   